# Homework 2: perturbation analysis of neoclassical growth model
## Computational Economics  (ECO309)

---

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__


Consider the deterministic neoclassical growth model

- Transition Equation
$$\begin{eqnarray}
k_t & = & (1-\delta) k_{t-1} + i_{t-1} \\\\
z_t & = & \rho z_{t-1}
\end{eqnarray}
$$

- Definition:
$$c_t = \exp(z_t) k_t^\alpha - i_t$$

- Control $c_t\in[0,\exp(z_t) k_t^\alpha[$

- Objective:
$$\max_{c_t} \sum_{t\geq0} \beta^t U(c_t)$$



- Optimality Condition:
$$\beta  \left[ \frac{\left(c_{t+1}\right)^{-\gamma}}{\left(c_t\right)^{-\gamma}} \left( (1-\delta) + \exp(z_{t+1} \alpha k_{t+1}^{\alpha -1} \right)\right] = 1$$
  - Takes into account the fact that $c_t>0$.
- Definition:
$$c_t = \exp(z_t) k_t^\alpha - i_t$$


- Calibration:
  - $\beta = 0.96$
  - $\delta = 0.1$
  - $\gamma = 4.0$
  - $\alpha = 0.3$
  - $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$
  

Our goal is to solve for the model dynamics close to the steady-state.

__Write down the lagrangian and derive the first order conditions (do not write the full derivation). You can assume that constraints on $c_t$ are never binding.__

__Create a structure `Calibration` to hold the model parameters.__

__Define two functions:__
- `transition(z::Number, k::Number, p::Calibration)::Tuple{Number}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z::Number, k::Number, c::Number, i::Number, Z::Number, K::Number, C::Number, I::Number, p::Calibration)::Number` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments. `s` and `x` represent the vector of states (z,k) and controls (c,i) respectively.__
- `arbitrage(s::AbstractVector, x::AbstractVector, S::AbstractVector, X::AbstractVector, p::Calibration)`
- `transition(s::AbstractVector, x::AbstractVector, p::Calibration)`

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns two vectors, one for the states, one for the controls, with the convention that lower case correspond to $t$ and upper case to $t+1$. Check that the steady-state $\overline{s}, \overline{x}$ satisfies the model equations, i.e. $\overline{s} = g(\overline{s}, \overline{x})$ and $f(\overline{s},\overline{x},\overline{s}, \overline{x})=0$.


The first order system satisfies:
$$\begin{eqnarray}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{eqnarray}$$
 
where $A, B, ...$ are obtained by differentiating the model at the steady-state w.r.t. the right argument.

For instance, to obtain `C`, one must compute the jacobian of $u\rightarrow f(\overline{s},\overline{x},u, \overline{x})$. Using ForwardDiff, it should be enough to do `ForwardDiff.jacobian(u->f(s0, x0, u, s0, x0, p), x0)`, where `s0` and `x0` are steady-state vectors for states and controls respectively.

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



__Write a function `first_order_model(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Matrix, M::PerturbedModel)` which computes the residual of this equation for a given 2x2 Matrix `X`.__


__Write a function `T(Y::Matrix, M::PerturbedModel)::Matrix`  which implements the time iteration step $X=-(B + C F + D Y F)^{-1}( A + C E + D Y E)$__

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods starting from initial state $s0$ (by using the formula $\Delta s_t = (E + F X) \Delta s_{t-1}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots. Show the convergence back to equilibrium when $k_0$ is 10\% lower than it's steady-state. Show the effect of a 1\% increase in productivity.__

---

__BONUS__

__Define the simulation matrix $P$ such that $s_{t+1} = P s_t$. Compute the spectral radius of `P`, using the power iteration method. Check that model is not diverging.__

__Define a a function `L( m::PerturbedModel,Y::Matrix, ΔY::Matrix)::Matrix` which applies the differential of T at Y to $\Delta Y$, i.e. which computes $T^{\prime}(Y).\Delta Y$. Note that the `.` operator denotes a linear operation (not necessarily a matrix multiplication). You can find hints, at the end of the slides on perturbations, to find how to compute it.__

__Use the power iteration method to compute the biggest eigenvalues of $T^{\prime}(X)$ where $X$ is obtained from linear time iteration. Check that Blanchard-Kahn conditions are met.__